### Change the files path if is necessary

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import os

from llama_index.core import Document
import optuna
import pandas as pd

from utils.retrieve import extract_question_ngrams

/home/dallan/pathway-indexer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dallan/pathway-indexer/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/dallan/pathway-indexer/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:843: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  r = torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [50]:
# configure
# filename = "test.md"
qa_filename = "index_single_quotes.csv"
ngram_size = 2  # use 2 instead of 3 so we don't skip 2-word header chunks
f_beta = 5  # weight recall 3 times as important as precision in f-score
n_trials = 100  # number of Optuna trials

pd.set_option("display.max_colwidth", None)


Read the question-answers


In [56]:
datapath = "../../data"

# qa_df = pd.read_csv(f'data/{qa_filename}', na_filter=False)
qa_df = pd.read_csv(f"{datapath}/temporary/{qa_filename}", na_filter=False)
print(len(qa_df))
qa_df.head(3)


98


,Question,Ideal Answer,Link,Quote
0,How do I know if a student has a scholarship?,"Missionaries can see if a student has a scholarship by clicking on the student's name in Path, clicking on the Discounts/Scholarships Tab and looking under History.\n\nInformation about the Discount or Scholarship includes; Discount/Scholarship, Category, Type, Percent, Amount, Term, Status.and Process.",https://missionaries.prod.byu-pathway.psdops.com/Finding-Student-Information-in-PATH%E2%80%8B,Discounts/Scholarships Tab\nView the scholarships and/or discounts the \nstudent has received.
1,How do I know if a student is registered for an institute class for credit?,"There is not a way for Missionaries to verify that a student's current Institute class, is registered for college credit for that class.\n\nMissionaries can see if a student is currently registered for an Institute class It should appear in the Enrollment tab in the lower right hand corner. There will be a red x indicating no credit if the course has not been completed. \nEach student needs to make sure that their Institute instructor knows and has recorded that the student is listed in the course as taking it for College Credit.\n\n",https://missionaries.prod.byu-pathway.psdops.com/institute-for-college-credit,"Checking for Institute Registration for College Credit\n2. Next, click on the Enrollment tab and scroll down\n4. You will see Institute courses taken by the student as you scroll. This\r\nstudent has taken many Institute courses but has not taken them for\r\ncollege credit. The student is currently enrolled in a Spring 2024 course but\r\nshows a x NO because the course is not yet completed. Once the course is\r\ncompleted, the x No will change to a green check mark with a Yes if the\r\ncourse qualifies for credit.\n5. This student is enrolled in an institute course for Spring 2024. It is not one\r\nof the cornerstone classes. The Student should ask the instructor if this\r\ncourse will provide college credit. If so, the No will change to Yes at the\r\ncompletion of the course. The previous 2023 course taken by this student has\r\nbeen given college credit"
2,How do I know if student is member of the church?,"Missionaries can see the Church membership status by clicking on the student's name in Path. On the Student Information Page, in the Details Tab, at the right side of the page, if there are Stake and a Ward names listed, the student is a member of the church.\n\nSometimes no Stake or Ward information will be listed, but when missionaries talk to the student they may find they are members after all. In that case, missionaries can ask the student if they have another Church Account. If they can't remember, they can search for a duplicate account. Students may have to get their membership number from the ward clerk to find the real church account.\n\n",https://missionaries.prod.byu-pathway.psdops.com/Verify-a-Learner's-Church-Member-Status,"Verify a Learner’s Membership Status\n1. In the Details tab in the Student Information page, verify that a learner’s\r\nward and stake are listed if they are a member of the church.\n2. A learner may show they are a member but have no ward and stake\r\nlisted. This is a good indicator that the learner has a duplicate account.\r\no Contact your zone or district leader to investigate this further."


In [57]:
# NOTE: we shouldn't include questions in the *test* set right now,
# but people are still adding the manual quotes,
# and since we have so few questions with manual quotes so far
# we will use all of them for this demo.

# keep only rows with at least 1 manual quote
qa_df = qa_df[qa_df["Quote"].notna() & (qa_df["Quote"] != "")].copy()
print(len(qa_df))


98


In [58]:
# generate bigrams (ngram size=2) for each manual quote
# and store them in the question_ngrams dictionary
question_ngrams = extract_question_ngrams(qa_df, ngram_size)
# store the ngrams in the dataframe
qa_df['question_ngrams'] = qa_df['Question'].map(question_ngrams)

Read the documents from the md directory


In [8]:
# Read the document names from the directories:
# ../data/markwodn
# ../data/out/md

origin_paths = [f"{datapath}/data_16_09_24/out/from_html/", f"{datapath}/data_16_09_24/out/md_files/"]
# origin_paths = [f"{datapath}/testdata/"]

# Read the document names from the directories:
files_list = [path + item for path in origin_paths for item in os.listdir(path)]


In [9]:
len(files_list)

539

In [10]:
files_list.sort()
files_list[0:10]


['../../data/data_16_09_24/out/from_html/105adb7a.md',
 '../../data/data_16_09_24/out/from_html/1376e9a9.md',
 '../../data/data_16_09_24/out/from_html/141279b1.md',
 '../../data/data_16_09_24/out/from_html/144bfd06.md',
 '../../data/data_16_09_24/out/from_html/14627860.md',
 '../../data/data_16_09_24/out/from_html/15c11982.md',
 '../../data/data_16_09_24/out/from_html/15ffa1be.md',
 '../../data/data_16_09_24/out/from_html/16ebd247.md',
 '../../data/data_16_09_24/out/from_html/179960e.md',
 '../../data/data_16_09_24/out/from_html/17e70845.md']

Save the documents into an array


In [11]:
documents = []

for i, filepath in enumerate(files_list):
    with open(filepath, "r", encoding="utf-8") as file:
        document = Document(text=file.read(), metadata={"filepath": filepath})

        # add the document to a single entry list
        documents.append(document)


In [12]:
len(documents)

539

In [13]:
documents[10]

Document(id_='721e76a0-95e5-4ae2-90a0-d24d906fe906', embedding=None, metadata={'filepath': '../../data/data_16_09_24/out/from_html/1abc5b56.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="---\nheading: Certificates & Degrees\nsubheading: Three-Year Degree\ntitle: Earn a Degree in Three Years\nurl: https://www.byupathway.edu/degree-in-three\n---\n# Earn a bachelor's degree in three years through BYU-Pathway Worldwide\n\n* Save Money\nFewer credits and guaranteed scholarships means extra savings\n* Finish Faster\nEarn a bachelor's degree one year ahead of schedule\n* Launch Your Career\nStart or upgrade your career sooner with a job-ready degree\n\n> To know that I could get done with school faster for less, was really what turned me to BYU-Pathway.\n> — Bryon Wensel, graduate\n\n## Fewer credits. 100% online.\n\nThrough BYU-Pathway Worldwide, BYU-Idaho and Ensign College offer 90–96 credit degrees, allowing students to finish an online bache

In [14]:
from collections.abc import Sequence
import re
import time
from typing import Any, Optional

import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.core.bridge.pydantic import Field
from llama_index.core.callbacks.base import CallbackManager
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser.interface import NodeParser
from llama_index.core.schema import BaseNode, MetadataMode, TextNode
from llama_index.core.utils import get_tqdm_iterable
# from llama_index.embeddings.voyageai import VoyageEmbedding
from llama_index.core.node_parser import (
    MarkdownNodeParser,
    SemanticSplitterNodeParser,
    SentenceSplitter,
)
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import spacy
import voyageai

from utils.retrieve import _generate_ngrams_from_texts, precision_recall, f_score
from utils.custom_node_parser import CustomNodeParser

vo = voyageai.Client()  # This will automatically use the environment variable VOYAGE_API_KEY.

In [15]:
def extract_index_metadata(node):
    """Split the document text into headers and content."""
    
    text = node.get_content(metadata_mode=MetadataMode.NONE)
    parts = text.split('---\n')
    
    # If there are no headers, return the document as-is
    if len(parts) < 3:
        return node
    
    # Extract headers
    headers_text = parts[1]
    
    # Parse headers into a dictionary
    headers = {}
    current_key = None
    for line in headers_text.strip().split('\n'):
        if ': ' in line:
            key, value = line.split(': ', 1)
            headers[key] = value
            current_key = key
        elif current_key is not None:
            # This line is a continuation of the previous value
            headers[current_key] += ' ' + line.strip()
            
    # clean up titles
    if 'title' in headers:
        title = headers['title']
        # Split the string using the corrected regex with escaped quotes
        titles = re.split(r"\s*['\"]?\s*,\s*['\"]?\s*", title)

        # Clean up each question by removing leading/trailing quotes and spaces
        titles = [re.sub(r'^[\'"\s]+|[\'"\s]+$', '', t) for t in titles]

        # Join the cleaned questions with the separator
        headers['title'] = " | ".join(titles)
              
    # Join the remaining parts as content
    content = '---\n'.join(parts[2:])
    
    return TextNode(metadata=headers, text=content)

In [16]:
documents = [extract_index_metadata(doc) for doc in documents]

In [17]:
metadata_keys = set()
for doc in documents:
    for key in doc.metadata:
        metadata_keys.add(key)
metadata_keys

{'heading', 'subheading', 'title', 'url'}

In [18]:
INDEX_METADATA_KEYS = ["heading", "subheading"]  # title often contains the question exactly and applies to only part of the document, "title"]
MD_METADATA_KEYS = ["header_1", "header_2", "header_3", "header_4", "header_5", "header_6"]
ORDERED_LIST_ITEM_PATTERN = r'^\s*\d+\.\s'
UNORDERED_LIST_ITEM_PATTERN = r'^\s*[-*+]\s'

def get_headers_and_paragraphs(node: BaseNode) -> list[str]:
    """Get headers and paragraphs from a node."""
    
    # get text from the node
    text = node.get_content(metadata_mode=MetadataMode.NONE)
    results = []
    paragraph_lines = []
    blank_line = False
    # split text by newline
    for line in text.split("\n"):
        line = line.strip()
        # is this a blank line? if so, add a newline to the paragraph
        if len(line) == 0:
            blank_line = True
        # is this a header? if so, add the previous paragraph to the results and this line to the results
        elif line.startswith("#"):
            if len(paragraph_lines) > 0:
                results.append("\n".join(paragraph_lines))
                paragraph_lines = []
            results.append(line)
            blank_line = False
        # add list items to the previous paragraph
        elif re.match(ORDERED_LIST_ITEM_PATTERN, line) or re.match(UNORDERED_LIST_ITEM_PATTERN, line):
            if blank_line and len(paragraph_lines) > 0:
                paragraph_lines.append("")
            paragraph_lines.append(line)
            blank_line = False
        else:
            # if this is a new paragraph, add the previous paragraph to the results
            if blank_line and len(paragraph_lines) > 0:
                results.append("\n".join(paragraph_lines))
                paragraph_lines = []
            paragraph_lines.append(line)
            blank_line = False
    if len(paragraph_lines) > 0:
        results.append("\n".join(paragraph_lines))
    return results


def update_headers(par: str, headers: dict) -> dict:
    """Set headers for a paragraph"""

    # Check if paragraph starts with a header (e.g. '# ', '## ', etc.)
    for level in range(1, 7):
        if par.startswith("#" * level + " "):
            headers[f"header_{level}"] = par.lstrip('# ')
            # Reset lower-level headers
            for lower_level in range(level + 1, 7):
                sub_header = f"header_{lower_level}"
                if sub_header in headers:
                    del headers[sub_header]
            # Exit loop after finding the correct header level
            break


def get_paragraph_nodes(headers_paragraphs: list[str], doc_node: BaseNode) -> list[TextNode]:
    """Get paragraph nodes with header metadata from a list of headers and paragraphs and the original document node."""
    
    nodes = []
    headers = {}
    doc_metadata={key: value for key, value in doc_node.metadata.items()}
    for par in headers_paragraphs:
        # if this is a header, update the headers
        if par.startswith("#"):
            update_headers(par, headers)
        else:
            metadata = {
                **headers,
                **doc_metadata,
                "context": par,
            }
            node = TextNode(metadata=metadata, text=par)
            nodes.append(node)
    return nodes


def _equal_headers(metadata1: dict, metadata2: dict) -> bool:
    """Check if two metadata dictionaries have the same headers."""
    
    for key in MD_METADATA_KEYS:
        if metadata1.get(key, "") != metadata2.get(key, ""):
            return False
    return True


def include_prev_next_contexts(nodes: list[TextNode], count: int, max_length: int) -> None:
    """
    Include up to count previous and next contexts in the context of each node 
    while they have the same headers and the combined length is less than the max length.
    """
    
    node_contexts = [node.metadata['context'] for node in nodes]
    for i, node in enumerate(nodes):
        node_context = node_contexts[i]
        for j in range(1, count + 1):
            prev_node = None if i - j < 0 else nodes[i - j]
            next_node = None if i + j >= len(nodes) else nodes[i + j]
            prev_context = ""
            next_context = ""
            if prev_node and _equal_headers(node.metadata, prev_node.metadata):
                prev_context = node_contexts[i - j]
            if next_node and _equal_headers(node.metadata, next_node.metadata):
                next_context = node_contexts[i + j]
            if len(prev_context) + len(node_context) + len(next_context) <= max_length:
                node_context = f"{prev_context}\n\n{node_context}\n\n{next_context}".strip()
            else:
                break
        node.metadata["context"] = node_context


def get_sentences(nodes: list[TextNode]) -> list[TextNode]:
    """Get sentence nodes from paragraph nodes."""
    
    nlp = spacy.load("en_core_web_sm")
    sentence_nodes = []
    for node in nodes:
        doc = nlp(node.text)
        for sent in doc.sents:
            metadata = {key: value for key, value in node.metadata.items()}
            sentence_nodes.append(TextNode(metadata=metadata, text=sent.text))
    return sentence_nodes


def embed_prev_next(nodes: list[TextNode], count: int, max_length: int) -> list[TextNode]:
    """
    Include up to count previous and next texts in the text of each node 
    while they have the same headers and the combined length is less than the max length.
    """
    
    embed_nodes = []
    for i, node in enumerate(nodes):
        node_text = node.text
        for j in range(1, count + 1):
            prev_node = None if i - j < 0 else nodes[i - j]
            next_node = None if i + j >= len(nodes) else nodes[i + j]
            prev_text = ""
            next_text = ""
            if prev_node and _equal_headers(node.metadata, prev_node.metadata):
                prev_text = prev_node.text
            if next_node and _equal_headers(node.metadata, next_node.metadata):
                next_text = next_node.text
            if len(prev_text) + len(node_text) + len(next_text) <= max_length:
                node_text = f"{prev_text}\n\n{node_text}\n\n{next_text}".strip()
            else:
                break
        node = TextNode(metadata={key: value for key, value in node.metadata.items()}, text=node_text)
        embed_nodes.append(node)
    return embed_nodes


def embed_metadata(nodes: list[TextNode], metadata_keys: list[str]):
    """Include metadata in the text of each node."""
    
    for node in nodes:
        headers = []
        for key in metadata_keys:
            value = node.metadata.get(key, "")
            if value:
                headers.append(value)
        if len(headers) > 0:
            node.text = f"{' / '.join(headers)}\n\n{node.text}"

        
def include_metadata(nodes: list[TextNode], metadata_keys: list[str]):
    """Include metadata in the context of each node."""
    
    for node in nodes:
        headers = []
        for key in metadata_keys:
            value = node.metadata.get(key, "")
            if value:
                headers.append(value)
        if len(headers) > 0:
            node.metadata["context"] = f"{' / '.join(headers)}\n\n{node.metadata.get('context', '')}"


class AltNodeParser(NodeParser):
    """Alternate Node Parser"""

    split_by: str = Field(
        default="para",
        description=(
            "Split by sentence, paragraph, or both."
        ),
    )

    embed_prev_next_sentences: int = Field(
        default=0,
        description=(
            "Number of previous and next sentences to include when calculating embeddings."
        ),
    )

    embed_prev_next_paragraphs: int = Field(
        default=0,
        description=(
            "Number of previous and next paragraphs to include when calculating embeddings."
        ),
    )

    max_embed_length: int = Field(
        default=2048,
        description=(
            "Maximum length of text to embed. Used when embedding previous and next sentences or paragraphs."
        ),
    )

    embed_index_headers: bool = Field(
        default=False,
        description=(
            "Include headers from the index page when calculating embeddings."
        ),
    )

    embed_md_headers: bool = Field(
        default=False,
        description=(
            "Include headers from the markdown document when calculating embeddings."
        ),
    )

    include_prev_next_paragraphs: int = Field(
        default=0,
        description=(
            "Number of previous and next paragraphs to include in text for the LLM."
        ),
    )

    max_include_length: int = Field(
        default=2048,
        description=(
            "Maximum length of text to pass to the LLM. Used when including previous and next paragraphs."
        ),
    )

    include_index_headers: bool = Field(
        default=False,
        description=(
            "Include headers from the index page in text for the LLM."
        ),
    )

    include_md_headers: bool = Field(
        default=False,
        description=(
            "Include headers from the markdown document in text for the LLM."
        ),
    )


    @classmethod
    def from_defaults(
        cls,
        split_by: str = "para",
        embed_prev_next_sentences: int = 0,
        embed_prev_next_paragraphs: int = 0,
        max_embed_length: int = 2048,
        embed_index_headers: bool = False,
        embed_md_headers: bool = False,
        include_prev_next_paragraphs: int = 0,
        max_include_length: int = 2048,
        include_index_headers: bool = False,
        include_md_headers: bool = False,
        callback_manager: Optional[CallbackManager] = None,
    ) -> "AltNodeParser":
        callback_manager = callback_manager or CallbackManager([])

        return cls(
            split_by=split_by,
            embed_prev_next_sentences=embed_prev_next_sentences,
            embed_prev_next_paragraphs=embed_prev_next_paragraphs,
            max_embed_length=max_embed_length,
            embed_index_headers=embed_index_headers,
            embed_md_headers=embed_md_headers,
            include_prev_next_paragraphs=include_prev_next_paragraphs,
            max_include_length=max_include_length,
            include_index_headers=include_index_headers,
            include_md_headers=include_md_headers,
        )

    @classmethod
    def class_name(cls) -> str:
        """Get class name."""
        return "AltNodeParser"

    def _parse_nodes(
        self,
        nodes: Sequence[BaseNode],
        show_progress: bool = False,
        **kwargs: Any,
    ) -> list[BaseNode]:
        all_nodes: list[BaseNode] = []
        nodes_with_progress = get_tqdm_iterable(nodes, show_progress, "Parsing nodes")

        for node in nodes_with_progress:
            nodes = self.get_nodes_from_node(node, **kwargs)
            all_nodes.extend(nodes)

        return all_nodes


    def get_nodes_from_node(self, node: BaseNode, **kwargs: Any) -> list[TextNode]:
        """Split a node into a list of nodes."""
        
        # turn node into a list of headers and paragraphs
        headers_paragraphs = get_headers_and_paragraphs(node)
        # get paragraph nodes (with metadata for headers and context text to send to the LLM)
        nodes = get_paragraph_nodes(headers_paragraphs, node)
        # include previous and next paragraphs
        if self.include_prev_next_paragraphs > 0:
            include_prev_next_contexts(nodes, self.include_prev_next_paragraphs, self.max_include_length)
        # split by sentence, paragraph, or both
        sentence_nodes = []
        if self.split_by in ["sentence", "both"]:
            sentence_nodes = get_sentences(nodes)
            if self.embed_prev_next_sentences > 0:
                sentence_nodes = embed_prev_next(sentence_nodes, self.embed_prev_next_sentences, self.max_embed_length)
        if self.split_by == "sentence":
            nodes = sentence_nodes
        else:  # paragraph or both
            if self.embed_prev_next_paragraphs > 0:
                nodes = embed_prev_next(nodes, self.embed_prev_next_paragraphs, self.max_embed_length)
            if self.split_by == "both":
                nodes.extend(sentence_nodes)
        # embed index and/or markdown headers
        if self.embed_index_headers or self.embed_md_headers:
            keys = []
            if self.embed_index_headers:
                keys.extend(INDEX_METADATA_KEYS)
            if self.embed_md_headers:
                keys.extend(MD_METADATA_KEYS)
            embed_metadata(nodes, keys)
        # include index and/or markdown headers in text sent to the LLM
        if self.include_index_headers or self.include_md_headers:
            keys = []
            if self.include_index_headers:
                keys.extend(INDEX_METADATA_KEYS)
            if self.include_md_headers:
                keys.extend(MD_METADATA_KEYS)
            include_metadata(nodes, keys)
        
        return nodes

In [19]:
ix = 10
doc = documents[ix]
doc

TextNode(id_='543dfac5-7fda-48df-a158-e807bca21b83', embedding=None, metadata={'heading': 'Certificates & Degrees', 'subheading': 'Three-Year Degree', 'title': 'Earn a Degree in Three Years', 'url': 'https://www.byupathway.edu/degree-in-three'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="# Earn a bachelor's degree in three years through BYU-Pathway Worldwide\n\n* Save Money\nFewer credits and guaranteed scholarships means extra savings\n* Finish Faster\nEarn a bachelor's degree one year ahead of schedule\n* Launch Your Career\nStart or upgrade your career sooner with a job-ready degree\n\n> To know that I could get done with school faster for less, was really what turned me to BYU-Pathway.\n> — Bryon Wensel, graduate\n\n## Fewer credits. 100% online.\n\nThrough BYU-Pathway Worldwide, BYU-Idaho and Ensign College offer 90–96 credit degrees, allowing students to finish an online bachelor’s degree in three years! The optimized degree program sa

In [20]:
get_headers_and_paragraphs(doc)

["# Earn a bachelor's degree in three years through BYU-Pathway Worldwide",
 "* Save Money\nFewer credits and guaranteed scholarships means extra savings\n* Finish Faster\nEarn a bachelor's degree one year ahead of schedule\n* Launch Your Career\nStart or upgrade your career sooner with a job-ready degree",
 '> To know that I could get done with school faster for less, was really what turned me to BYU-Pathway.\n> — Bryon Wensel, graduate',
 '## Fewer credits. 100% online.',
 'Through BYU-Pathway Worldwide, BYU-Idaho and Ensign College offer 90–96 credit degrees, allowing students to finish an online bachelor’s degree in three years! The optimized degree program saves you time and money by eliminating elective credits.',
 '### Available degrees',
 '* [Applied Business Management](https://www.byupathway.edu/bachelors-degree/applied-business-management)\n* [Information Technology](https://www.byupathway.edu/bachelors-degree/information-technology)\n* [Applied Health](https://www.byupathwa

In [21]:
parser = AltNodeParser.from_defaults()
parser.get_nodes_from_node(doc)

[TextNode(id_='25f45a9d-e2fc-4002-b6e3-15d1cd76dccc', embedding=None, metadata={'header_1': "Earn a bachelor's degree in three years through BYU-Pathway Worldwide", 'heading': 'Certificates & Degrees', 'subheading': 'Three-Year Degree', 'title': 'Earn a Degree in Three Years', 'url': 'https://www.byupathway.edu/degree-in-three', 'context': "* Save Money\nFewer credits and guaranteed scholarships means extra savings\n* Finish Faster\nEarn a bachelor's degree one year ahead of schedule\n* Launch Your Career\nStart or upgrade your career sooner with a job-ready degree"}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="* Save Money\nFewer credits and guaranteed scholarships means extra savings\n* Finish Faster\nEarn a bachelor's degree one year ahead of schedule\n* Launch Your Career\nStart or upgrade your career sooner with a job-ready degree", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}'

In [22]:
def run_pipeline(documents, splitter, embed_model, vector_store, include_prev_next_rel):
    """Run the ingestion pipeline to split documents, generate embeddings, and insert into an index."""
    pipeline = IngestionPipeline(
        transformations=[
            splitter,
            embed_model,
        ]
    )
    index = VectorStoreIndex.from_vector_store(
        vector_store,
        embed_model=embed_model,
    )
    nodes = pipeline.run(documents=documents, show_progress=False)
    
    if include_prev_next_rel:
        for i in range(0, len(nodes)):
            if i > 0:
                nodes[i].metadata["prev"] = nodes[i - 1].text
            if i < len(nodes) - 1:
                nodes[i].metadata["next"] = nodes[i + 1].text
                
    index.insert_nodes(nodes)
    print(f"Nodes inserted: {len(nodes)}")
    return index, nodes  # CHANGED

In [23]:
def evaluate_retriever(retriever, 
                       qa_df, 
                       ngram_size, 
                       f_beta, 
                       retriever_threshold=0.0, 
                       rerank=False,
                       rerank_model="",
                       rerank_threshold=0.0,
                       rerank_k=0,
                       include_prev_next_rel=False, 
                       use_node_contexts=False
                      ):
    """Evaluate the retriever using a set of questions and their corresponding n-grams."""
    f_scores = []
    results = []
    for index, row in qa_df.iterrows():
        question = row['Question']
        true_ngrams = row['question_ngrams']
        response = retriever.retrieve(question)
        
        # CHANGED - set a minimum similarity threshold
        nodes = [node for node in response if node.score >= retriever_threshold]
        
        # CHANGED - use node context metadata for text to send to llm
        if use_node_contexts:
            node_texts_ordered = [node.metadata.get("context", "") for node in nodes]
        elif include_prev_next_rel:
            node_texts_ordered = [
                node.metadata.get("prev", "") + " " + node.text + " " + node.metadata.get("next", "")
                for node in nodes
            ]
        else:
            node_texts_ordered = [node.text for node in nodes]
            
        # CHANGED - dedup node texts
        node_texts = list(set(node_texts_ordered))
        
        # CHANGED - rerank
        if rerank and len(node_texts) > 1:
            success = False
            retries = 0
            while not success and retries < 3:
                try:
                    reranking = vo.rerank(question, node_texts, model=rerank_model, top_k=rerank_k)
                    node_texts = [r.document for r in reranking.results if r.relevance_score >= rerank_threshold]            
                    success = True
                except:
                    time.sleep(60)
                    retries += 1
        
        # CHANGED - ensure node texts is not empty
        node_text_nodes = []
        if len(node_texts) == 0:
            node_texts = ['qwer asdf']
        else:
            # CHANGED - get filtered and reranked nodes
            for node_text in node_texts:
                node_text_nodes.append((node_text, nodes[node_texts_ordered.index(node_text)]))
            
        predicted_ngrams = _generate_ngrams_from_texts(node_texts, ngram_size=ngram_size)
        # CHANGED - dedup ngrams
        true_ngrams = list(set(true_ngrams))
        predicted_ngrams = list(set(predicted_ngrams))
        precision, recall = precision_recall(predicted_ngrams, true_ngrams)
        score = f_score(precision, recall, beta=f_beta)
        f_scores.append(score)
        results.append({
            "Question": question,
            "Ideal Answer": row['Ideal Answer'],
            "Link": row['Link'],
            'Quote': row['Quote'],
            'Chunk urls': '\n'.join(list(set(node.metadata.get('url', '') for _, node in node_text_nodes))),
            'Chunks': '\n\n\n'.join([f"{node.metadata.get('url', '')}\n-----\n{node_text}" for node_text, node in node_text_nodes]),
            'Precision': precision,
            'Recall': recall,
            f'F{f_beta}': score,
        })
            
    return sum(f_scores) / len(f_scores), pd.DataFrame(results)

In [24]:
def objective(trial, documents, ngram_size, qa_df, f_beta=1.0):
    """
    This function is called by Optuna. It creates an index, run queries over the index,
    calculates the precision and recall of the results, and returns the average f-score.
    """

    #
    # Define hyperparameters
    #

    # define embedder
    embed_model_name = trial.suggest_categorical(
        "embed_model",
        [
            # "text-embedding-3-small",
            "text-embedding-3-large",  # .02 better, 6x more expensive
            # we can't use voyage embeddings right now because the version of llamaindex that we're using 
            # uses and older version of the voyage client that doesn't support re-ranking
            # "voyage-3-lite",  # .01 better, same price as text-embedding-3-small
            # "voyage-3",  # worse
            # GPU runs out of memory with gte model
            # 'Alibaba-NLP/gte-large-en-v1.5',  # WARNING: this downloads 1.74G of data
        ],
    )
    if embed_model_name in ["text-embedding-3-small", "text-embedding-3-large"]:
        embed_model = OpenAIEmbedding(
            model=embed_model_name,
            embed_batch_size=100,
            max_retries=25,
            timeout=180,
            reuse_client=True,   
        )
    elif embed_model_name in ["voyage-3", "voyage-3-lite"]:
        embed_model = VoyageEmbedding(
            model_name=embed_model_name
        )

    # define splitter
    splitter_name = trial.suggest_categorical(
        "splitter",
        [
            # "sentence",
            # "semantic",
            # "markdown",
            # "paragraph"
            # "custom_splitter",
            "alt_splitter",
        ],
    )

    if splitter_name == "sentence":
        chunk_size = trial.suggest_int("chunk_size", 256, 1024)
        chunk_overlap = trial.suggest_int("chunk_overlap", 0, 200)
        splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    elif splitter_name == "semantic":
        include_prev_next_rel = trial.suggest_categorical("include_prev_next_rel", [True, False])  # CHANGE
        buffer_size = trial.suggest_int("buffer_size", 1, 3)
        breakpoint_percentile_threshold = trial.suggest_int("breakpoint_percentile_threshold", 60, 85)
        splitter = SemanticSplitterNodeParser(
            buffer_size=buffer_size,
            breakpoint_percentile_threshold=breakpoint_percentile_threshold,
            include_prev_next_rel=include_prev_next_rel,
            embed_model=embed_model,
        )

    elif splitter_name == "markdown":
        include_prev_next_rel = trial.suggest_categorical("include_prev_next_rel", [True, False])  # CHANGE
        splitter = MarkdownNodeParser(
            include_prev_next_rel=include_prev_next_rel,
        )
    # elif splitter_name == "paragraph":
    #     splitter = split_document_text

    elif splitter_name == "custom_splitter":
        add_metadata_to_text = trial.suggest_categorical("add_metadata_to_text", [True])
        split_by_sentence = trial.suggest_categorical("split_by_sentence", [True])
        include_prev_next_rel = trial.suggest_categorical("include_prev_next_rel", [True])  # CHANGE
        splitter = CustomNodeParser().from_defaults(
            add_metadata_to_text=add_metadata_to_text, split_by_sentence=split_by_sentence
        )
    elif splitter_name == "alt_splitter":  # CHANGED
        split_by = "paragraph"  ### trial.suggest_categorical("split_by", ["sentence", "paragraph", "both"])
        embed_prev_next_sentences = 0  ### trial.suggest_int("embed_prev_next_sentences", 0, 5)
        embed_prev_next_paragraphs = 1  ### trial.suggest_int("embed_prev_next_paragraphs", 1, 4)
        max_embed_length = trial.suggest_int("max_embed_length", 350, 500, step=50)
        # Two problems with embedding index headers:
        # 1. The index titles are often the exact same as our test questions, leading to data leakage 
        #    which makes our numbers too good.
        # 2. The index titles may refer to specific parts of the page, but we associate them with the entire page
        #    which means that every chunk on the page appears equally relevant and gets our retriever confused.
        # So I think we'll have to rely on embedding markdown headers
        embed_index_headers = trial.suggest_categorical("embed_index_headers", [True, False])
        embed_md_headers = True  ### trial.suggest_categorical("embed_md_headers", [True])
        include_prev_next_paragraphs = trial.suggest_int("include_prev_next_paragraphs", 2, 5)
        max_include_length = trial.suggest_int("max_include_length", 650, 800, step=50)
        include_index_headers = False  ### trial.suggest_categorical("include_index_headers", [False])
        include_md_headers = True  ### trial.suggest_categorical("include_md_headers", [True])
        splitter = AltNodeParser().from_defaults(
            split_by=split_by,
            embed_prev_next_sentences=embed_prev_next_sentences,
            embed_prev_next_paragraphs=embed_prev_next_paragraphs,
            max_embed_length=max_embed_length,
            embed_index_headers=embed_index_headers,
            embed_md_headers=embed_md_headers,
            include_prev_next_paragraphs=include_prev_next_paragraphs,
            max_include_length=max_include_length,
            include_index_headers=include_index_headers,
            include_md_headers=include_md_headers,
        )
        
    # add metadata
    ## nothing for now

    # define index
    query_mode = VectorStoreQueryMode.DEFAULT
    index_type = trial.suggest_categorical(
        "index",
        [
            "chromadb",
            # "milvus",  # need to create a (free) account at https://cloud.zilliz.com/
            # and add MILVUS_URI=your public endpoint and MILVUS_TOKEN=your token (api key) to your .env file
        ],
    )
    if index_type == "chromadb":
        chroma_client = chromadb.EphemeralClient()
        # delete collection if it exists
        if any(coll.name == "test" for coll in chroma_client.list_collections()):
            chroma_client.delete_collection("test")
        chroma_collection = chroma_client.create_collection("test")
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

    # define retriever hyperparameters
    retriever_threshold = trial.suggest_float("retriever_threshold", 0.0, 0.15, step=0.05)  # CHANGED
    retriever_k = trial.suggest_int("retriever_k", 35, 40, step=5)  # CHANGED 
    sparse_k = retriever_k * 5

    # define reranker hyperparameters
    rerank = trial.suggest_categorical("rerank", [True])
    if rerank:
        rerank_model = "rerank-lite-1"
        rerank_threshold = trial.suggest_float("rerank_threshold", 0.2, 0.4, step=0.02)
        rerank_k = trial.suggest_int("rerank_k", 16, 18, step=1)
    else:
        rerank_model = ""
        rerank_threshold = 0.0
        rerank_k = 0
        
    #
    # Use hyperparameters to split documents into chunks, generate embeddings, and insert into an index
    #

    # run the pipeline
    index, nodes = run_pipeline(documents, splitter, embed_model, vector_store, 
                                splitter_name in ["semantic", "markdown", "custom_splitter"] and include_prev_next_rel)

    # create a retriever from the index
    retriever = index.as_retriever(
        vector_store_query_mode=query_mode,
        similarity_top_k=retriever_k,
        sparse_top_k=sparse_k,
    )
    
    #
    # Evaluate the quality of the chunks retrieved from the index for the sample questions
    #

    # issue all questions and calculate the f-score on the retrieved chunks
    avg_f_score, eval_df = evaluate_retriever(
        retriever, 
        qa_df, 
        ngram_size, 
        f_beta, 
        retriever_threshold=retriever_threshold,
        rerank=rerank,
        rerank_model=rerank_model,
        rerank_threshold=rerank_threshold,
        rerank_k=rerank_k,
        include_prev_next_rel=splitter_name in ["semantic", "markdown", "custom_splitter"] and include_prev_next_rel,
        use_node_contexts=splitter_name == "alt_splitter",
    )
    
    # write nodes
    nodes_df = pd.DataFrame([{
        'url': node.metadata.get('url', ''),
        'text': node.text,
        'context': node.metadata.get('context', ''),
    } for node in nodes])
    nodes_df.to_csv(f'chunks_{trial.number}.csv', index=False)

    # write eval_df
    eval_df.to_csv(f'chunks_recall_{trial.number}.csv', index=False)
                    
    return avg_f_score


In [25]:
# ask Optuna to find the best hyperparameters

study_name = "test_09_22_24_ih"  # Unique identifier of the study.
storage_name = f"sqlite:///optuna-{study_name}.db"
print(
    f"To see a dashboard, open a terminal, activate the virtual environment, and run: optuna-dashboard {storage_name}"
)
study = optuna.create_study(
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
    direction="maximize",
)
# study.optimize(
#     get_objective_fn(
#         documents=documents, ngram_size=ngram_size, question_ngrams=question_ngrams
#     ),
#     n_trials=n_trials,
# )

study.optimize(
    lambda trial: objective(
        trial, documents, ngram_size, qa_df, f_beta=f_beta
    ),
    n_trials=n_trials,
)


To see a dashboard, open a terminal, activate the virtual environment, and run: optuna-dashboard sqlite:///optuna-test_09_22_24_ih.db


[I 2024-09-22 20:40:36,282] A new study created in RDB with name: test_09_22_24_ih


Nodes inserted: 8121


[I 2024-09-22 20:47:06,257] Trial 0 finished with value: 0.4678342691281568 and parameters: {'embed_model': 'text-embedding-3-large', 'splitter': 'alt_splitter', 'max_embed_length': 450, 'embed_index_headers': False, 'include_prev_next_paragraphs': 3, 'max_include_length': 750, 'index': 'chromadb', 'retriever_threshold': 0.1, 'retriever_k': 40, 'rerank': True, 'rerank_threshold': 0.36, 'rerank_k': 16}. Best is trial 0 with value: 0.4678342691281568.


Nodes inserted: 8121


...

Nodes inserted: 8121


[I 2024-09-23 07:11:04,806] Trial 94 finished with value: 0.49114510245589776 and parameters: {'embed_model': 'text-embedding-3-large', 'splitter': 'alt_splitter', 'max_embed_length': 400, 'embed_index_headers': False, 'include_prev_next_paragraphs': 3, 'max_include_length': 750, 'index': 'chromadb', 'retriever_threshold': 0.05, 'retriever_k': 35, 'rerank': True, 'rerank_threshold': 0.2, 'rerank_k': 18}. Best is trial 84 with value: 0.5093350790002985.


Nodes inserted: 8121


[I 2024-09-23 07:17:36,018] Trial 95 finished with value: 0.4839495911368331 and parameters: {'embed_model': 'text-embedding-3-large', 'splitter': 'alt_splitter', 'max_embed_length': 400, 'embed_index_headers': False, 'include_prev_next_paragraphs': 3, 'max_include_length': 800, 'index': 'chromadb', 'retriever_threshold': 0.05, 'retriever_k': 35, 'rerank': True, 'rerank_threshold': 0.22, 'rerank_k': 18}. Best is trial 84 with value: 0.5093350790002985.
[W 2024-09-23 07:19:48,811] Trial 96 failed with parameters: {'embed_model': 'text-embedding-3-large', 'splitter': 'alt_splitter', 'max_embed_length': 400, 'embed_index_headers': False, 'include_prev_next_paragraphs': 3, 'max_include_length': 750, 'index': 'chromadb', 'retriever_threshold': 0.05, 'retriever_k': 35, 'rerank': True, 'rerank_threshold': 0.2, 'rerank_k': 18} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/dallan/pathway-indexer/.venv/lib/python3.12/site-packages/optuna/st

KeyboardInterrupt: 

In [ ]:
study.best_params